In [1]:
import datetime
import numpy as np
import pandas as pd
import requests

def get_binance_data(ticker, interval, start, end):
  columns = ['open_time','open', 'high', 'low', 'close', 'volume','close_time', 'qav','num_trades','taker_base_vol','taker_quote_vol', 'ignore']
  usecols=['open', 'high', 'low', 'close', 'volume', 'qav','num_trades','taker_base_vol','taker_quote_vol']

  start = int(datetime.datetime.timestamp(pd.to_datetime(start))*1000)
  end_u = int(datetime.datetime.timestamp(pd.to_datetime(end))*1000)

  df = pd.DataFrame()
 
  while True:
    url = f'https://www.binance.com/api/v3/klines?symbol={ticker}&interval={interval}&limit=1000&startTime={start}#&endTime={end_u}'
    data = pd.DataFrame(requests.get(url).json(), columns=columns, dtype=np.float64)    
    if data.empty:
      break
    start = int(data.open_time.tolist()[-1])+1
    data.index = [pd.to_datetime(x, unit='ms') for x in data['open_time']]
    df = pd.concat([df, data], axis=0)
    if end in data.index.tolist():
      break
 
  df.index = pd.to_datetime(df.index)
  df = df.loc[:end]
  df['coin price'] = df['close']

  return df[['coin price']]

def get_weekly_returns(coin, weekly_investment_usd):
    df = get_binance_data(ticker=f'{coin}USDT', interval='1w', start='2020-01-01T00:00:00', end='2022-01-01T00:00:00')    
    df['week'] = range(1, len(df) + 1)
    df['USD investment'] = weekly_investment_usd
    df['USD total'] = df['USD investment'].cumsum()
    df['coins bought'] = weekly_investment_usd / df['coin price']
    df['coins total'] = df['coins bought'].cumsum()
    df['USD return'] = df['coins total'] * df['coin price']
    df['USD profit'] = df['USD return'] - df['USD total']

    return df[['week', 'coin price', 'USD total', 'coins bought', 'coins total', 'USD return', 'USD profit']] \
      .style.format({
        "USD investment": "${:,.0f}",
        "USD total": "${:,.0f}",
        "USD profit": "${:,.0f}",
        "USD return": "${:,.0f}"
       }) \
      .highlight_max(color='green') \
      .highlight_min(color='#cd4f39')

In [2]:
get_weekly_returns('BTC', weekly_investment_usd=25)

,week,coin price,USD total,coins bought,coins total,USD return,USD profit
2020-01-06 00:00:00,1,8184.980000,$25,0.003054,0.003054,$25,$0
2020-01-13 00:00:00,2,8701.700000,$50,0.002873,0.005927,$52,$2
2020-01-20 00:00:00,3,8615.000000,$75,0.002902,0.008829,$76,$1
2020-01-27 00:00:00,4,9331.510000,$100,0.002679,0.011508,$107,$7
2020-02-03 00:00:00,5,10151.750000,$125,0.002463,0.013971,$142,$17
2020-02-10 00:00:00,6,9917.270000,$150,0.002521,0.016492,$164,$14
2020-02-17 00:00:00,7,9936.400000,$175,0.002516,0.019008,$189,$14
2020-02-24 00:00:00,8,8531.880000,$200,0.002930,0.021938,$187,$-13
2020-03-02 00:00:00,9,8033.310000,$225,0.003112,0.025050,$201,$-24
2020-03-09 00:00:00,10,5361.300000,$250,0.004663,0.029713,$159,$-91


In [3]:
get_weekly_returns('ETH', weekly_investment_usd=25)

,week,coin price,USD total,coins bought,coins total,USD return,USD profit
2020-01-06 00:00:00,1,146.540000,$25,0.170602,0.170602,$25,$0
2020-01-13 00:00:00,2,166.790000,$50,0.149889,0.320491,$53,$3
2020-01-20 00:00:00,3,167.860000,$75,0.148934,0.469425,$79,$4
2020-01-27 00:00:00,4,188.440000,$100,0.132668,0.602093,$113,$13
2020-02-03 00:00:00,5,228.490000,$125,0.109414,0.711507,$163,$38
2020-02-10 00:00:00,6,258.850000,$150,0.096581,0.808088,$209,$59
2020-02-17 00:00:00,7,274.480000,$175,0.091081,0.899169,$247,$72
2020-02-24 00:00:00,8,217.810000,$200,0.114779,1.013948,$221,$21
2020-03-02 00:00:00,9,199.430000,$225,0.125357,1.139305,$227,$2
2020-03-09 00:00:00,10,123.780000,$250,0.201971,1.341277,$166,$-84


In [4]:
get_weekly_returns('SOL', weekly_investment_usd=25)

,week,coin price,USD total,coins bought,coins total,USD return,USD profit
2020-08-10 00:00:00,1,3.388500,$25,7.377896,7.377896,$25,$0
2020-08-17 00:00:00,2,3.250000,$50,7.692308,15.070204,$49,$-1
2020-08-24 00:00:00,3,4.430900,$75,5.642195,20.712398,$92,$17
2020-08-31 00:00:00,4,3.094100,$100,8.079894,28.792292,$89,$-11
2020-09-07 00:00:00,5,3.260300,$125,7.668006,36.460298,$119,$-6
2020-09-14 00:00:00,6,2.862800,$150,8.732709,45.193007,$129,$-21
2020-09-21 00:00:00,7,3.157800,$175,7.916904,53.109912,$168,$-7
2020-09-28 00:00:00,8,2.520200,$200,9.919848,63.029759,$159,$-41
2020-10-05 00:00:00,9,2.422700,$225,10.319066,73.348825,$178,$-47
2020-10-12 00:00:00,10,2.215000,$250,11.286682,84.635506,$187,$-63


In [5]:
get_weekly_returns('MATIC', weekly_investment_usd=25)

,week,coin price,USD total,coins bought,coins total,USD return,USD profit
2020-01-06 00:00:00,1,0.014960,$25,1671.122995,1671.122995,$25,$0
2020-01-13 00:00:00,2,0.016430,$50,1521.606817,3192.729811,$52,$2
2020-01-20 00:00:00,3,0.016450,$75,1519.756839,4712.486650,$78,$3
2020-01-27 00:00:00,4,0.019030,$100,1313.715187,6026.201837,$115,$15
2020-02-03 00:00:00,5,0.021970,$125,1137.915339,7164.117176,$157,$32
2020-02-10 00:00:00,6,0.020110,$150,1243.162606,8407.279782,$169,$19
2020-02-17 00:00:00,7,0.022290,$175,1121.579183,9528.858965,$212,$37
2020-02-24 00:00:00,8,0.019350,$200,1291.989664,10820.848629,$209,$9
2020-03-02 00:00:00,9,0.021430,$225,1166.588894,11987.437523,$257,$32
2020-03-09 00:00:00,10,0.009860,$250,2535.496957,14522.934481,$143,$-107


In [6]:
get_weekly_returns('DOGE', weekly_investment_usd=25)

,week,coin price,USD total,coins bought,coins total,USD return,USD profit
2020-01-06 00:00:00,1,0.002196,$25,11382.780130,11382.780130,$25,$0
2020-01-13 00:00:00,2,0.002398,$50,10424.485030,21807.265161,$52,$2
2020-01-20 00:00:00,3,0.002285,$75,10939.004113,32746.269274,$75,$-0
2020-01-27 00:00:00,4,0.002441,$100,10243.382775,42989.652048,$105,$5
2020-02-03 00:00:00,5,0.003223,$125,7755.545215,50745.197263,$164,$39
2020-02-10 00:00:00,6,0.002752,$150,9083.642177,59828.839440,$165,$15
2020-02-17 00:00:00,7,0.002677,$175,9338.114448,69166.953888,$185,$10
2020-02-24 00:00:00,8,0.002279,$200,10967.798543,80134.752432,$183,$-17
2020-03-02 00:00:00,9,0.002167,$225,11536.154308,91670.906739,$199,$-26
2020-03-09 00:00:00,10,0.001693,$250,14764.942121,106435.848861,$180,$-70
